# Mancala Game

Names: Alex Schwarz, Rey Stone

---

### [Official GitHub repo](https://github.com/exkcd/mancala-game/tree/main)

## DISCLAIMER

We have installed a progress bar library called `tqdm` for additional prettificaiton. It will be uncommented, but if you want to install it the instructions are in our GitHub repo

## Mancala rules being implemented

**There are many different rules sets for Mancala. Please read this before writing the code:**

- Players sit on opposite sides of the long edge of the board
- There are 6 small pits in the middle of the board and 2 large ones at each end. The small ones in the middle and the large pit on your right are yours. The small ones on the other side and the large pit to your opponent's right are theirs
- The large pits at the end of the board are called Mancalas
- Set up the board with 4 stones per small pit (none in the mancalas)
- On every turn, select a pit on your side of the board that contains one or more stones, then distribute its stones, one stone per pit, in an counter-clockwise direction until you have no stones remaining
- If you encounter your opponent's mandala, skip it
- If you encounter your mancala, drop a stone into it
- If the last stone lands in an empty pit on your side of the board, capture this stone and any stones in your opponent's pit on the other side of the board, collect all of these stones, including the one that just landed, and place them into your mancala.
- If either player's pits are entirely empty, the game concludes.
- The player who still has stones on his side of the board when the game concludes places all of these pieces into their mancala.
  The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.

## Min-max algroithm

In [1]:
import numpy as np


def minmax_decision(game, depth=4):

    player = game.current_player

    def max_value(depth):
        if depth == 0 or game.winning_eval():
            return game.utility(game.board, player)

        v = -np.inf
        for a in game.actions():
            undo_info = game.play_with_undo(a)
            v = max(v, min_value(depth - 1))
            game.undo_move(undo_info)
        return v

    def min_value(depth):
        if depth == 0 or game.winning_eval():
            return game.utility(game.board, player)

        v = np.inf
        for a in game.actions():
            undo_info = game.play_with_undo(a)
            v = min(v, max_value(depth - 1))
            game.undo_move(undo_info)
        return v

    best_score = -np.inf
    best_action = None

    for a in game.actions():
        undo_info = game.play_with_undo(a)
        value = min_value(depth - 1)
        game.undo_move(undo_info)

        if value > best_score:
            best_score = value
            best_action = a

    return best_action


## Alpha-beta algorithm

In [2]:
import numpy as np


def alpha_beta_search(game, depth=4):
    player = game.current_player

    def max_value(alpha, beta, depth):
        if depth == 0 or game.winning_eval():
            return game.utility(game.board, player)

        v = -np.inf
        for a in game.actions():
            undo_info = game.play_with_undo(a)
            v = max(v, min_value(alpha, beta, depth-1))
            game.undo_move(undo_info)

            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(alpha, beta, depth):
        if depth == 0 or game.winning_eval():
            return game.utility(game.board, player)

        v = np.inf
        for a in game.actions():
            undo_info = game.play_with_undo(a)
            v = min(v, max_value(alpha, beta, depth-1))
            game.undo_move(undo_info)

            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    best_score = -np.inf
    alpha = -np.inf
    beta = np.inf
    best_action = None

    for a in game.actions():
        undo_info = game.play_with_undo(a)
        v = min_value(alpha, beta, depth-1)
        game.undo_move(undo_info)

        if v > best_score:
            best_score = v
            best_action = a
            alpha = v

    return best_action


## Formatting stuff

In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def stat_title(title, buffer):
    print(f"{color.GREEN + color.BOLD}⊰{'Ξ' * buffer}{title}{'Ξ' * buffer}⊱{color.END}")
    #Unicode U+22B1 character for the ⊱ symbol and greek symbol Xi for Ξ which is U+039E

def list_stat(name, stat, int):
    print(f"{name}{stat.rjust(int, ' ')}")


## Base Mancala Game

In [4]:
import random
from utilities.formatting import color

class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit=4, print_output=True):
        self.pits_per_player = pits_per_player
        # Initialize each pit with stones_per_pit number of stones
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [
            self.pits_per_player + 1, len(self.board) - 1 - 1]
        self.p2_mancala_index = len(self.board) - 1

        self.p1_win = 0
        self.p2_win = 0
        self.draw = 0
        self.print_output = print_output

        # winning advantage
        self.first= 0
        self.wins_w_first= 0

        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i],
                                                       player_2_pits[-(i + 1)], self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i],
                                                       player_2_pits[-(i + 1)], self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))

    def valid_move(self, pit): # actions
        return True if pit in range(1, self.pits_per_player+1) else False
    
    def actions(self):
        valid_pits = []
        if self.current_player == 1:
            for pit in range(1, self.pits_per_player + 1):
                board_index = pit - 1
                if self.board[board_index] > 0:
                    valid_pits.append(pit)
        else:
            for pit in range(1, self.pits_per_player + 1):
                board_index = self.p2_pits_index[0] + (pit - 1)
                if self.board[board_index] > 0:
                    valid_pits.append(pit)
        return valid_pits

    def random_move_generator(self):
        valid_pits = []
        if self.current_player == 1:
            for pit in range(1, self.pits_per_player + 1):
                board_index = pit - 1
                if self.board[board_index] > 0:
                    valid_pits.append(pit)
        else:
            for pit in range(1, self.pits_per_player + 1):
                board_index = self.p2_pits_index[0] + (pit - 1)
                if self.board[board_index] > 0:
                    valid_pits.append(pit)
        if valid_pits:
            return random.choice(valid_pits)
        return None

    def play(self, pit): # result
        if pit is None:
            return self.board
        
        if not self.winning_eval():
            if self.print_output:
                self.print_moves(pit)
            current_index = self.get_pit_index(pit)

            if not self.valid_move(pit):
                if self.print_output:
                    print("INVALID MOVE")
                return self.board

            stones = self.board[current_index]  # get amount of stones
            self.board[current_index] = 0  # remove stones
            opponent_mancala = self.p2_mancala_index if self.current_player == 1 else self.p1_mancala_index

            while stones > 0:
                current_index = (current_index + 1) % len(self.board)

                if current_index == opponent_mancala:
                    continue
                self.board[current_index] += 1
                stones -= 1

            self.capture_stones(current_index)
            self.moves.append((self.current_player, pit))
            self.switch_player()
        return self.board

    def capture_stones(self, current_index):
        if self.current_player == 1:
            if (current_index >= self.p1_pits_index[0] and
                current_index <= self.p1_pits_index[1] and
                    self.board[current_index] == 1):
                opposite_index = self.p2_pits_index[1] - \
                    (current_index - self.p1_pits_index[0])
                if self.board[opposite_index] > 0:
                    captured = self.board[opposite_index] + \
                        self.board[current_index]
                    self.board[opposite_index] = 0
                    self.board[current_index] = 0
                    self.board[self.p1_mancala_index] += captured
        else:
            if (current_index >= self.p2_pits_index[0] and
                current_index <= self.p2_pits_index[1] and
                    self.board[current_index] == 1):
                opposite_index = self.p1_pits_index[1] - \
                    (current_index - self.p2_pits_index[0])
                if self.board[opposite_index] > 0:
                    captured = self.board[opposite_index] + \
                        self.board[current_index]
                    self.board[opposite_index] = 0
                    self.board[current_index] = 0
                    self.board[self.p2_mancala_index] += captured

    def print_moves(self, pit):
        if self.current_player == 1:
            print(
                f'{color.BLUE}Player {self.current_player} chose pit: {pit}{color.END}')
        else:
            print(
                f'{color.RED}Player {self.current_player} chose pit: {pit}{color.END}')

    def get_pit_index(self, pit):
        if self.current_player == 1:
            return pit - 1
        else:
            return self.p1_mancala_index + pit

    def switch_player(self):
        self.current_player = 2 if self.current_player == 1 else 1

    def check_win(self):
        if self.winning_eval():
            p1_total = self.board[self.p1_mancala_index]
            p2_total = self.board[self.p2_mancala_index]
            if p1_total > p2_total:
                self.p1_win = 1
                if(self.first == 1):
                    self.wins_w_first += 1
                if self.print_output:
                    print(f'{color.BOLD + color.BLUE}GAME OVER: P1 wins!{color.END}')
            elif p2_total > p1_total:
                self.p2_win = 1
                if(self.first == 2):
                    self.wins_w_first = 1
                if self.print_output:
                    print(f'{color.BOLD + color.RED}GAME OVER: P2 wins!{color.END}')
            elif p1_total == p2_total:
                self.draw = 1
                if self.print_output:
                    print(f'{color.BOLD + color.YELLOW}GAME OVER: It\'s a draw!{color.END}')

    def winning_eval(self):
        p1_empty = all(self.board[i] == 0 for i in range(
            self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_empty = all(self.board[i] == 0 for i in range(
            self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        if p1_empty or p2_empty:
            if not p1_empty:
                for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                    self.board[self.p1_mancala_index] += self.board[i]
                    self.board[i] = 0
            if not p2_empty:
                for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                    self.board[self.p2_mancala_index] += self.board[i]
                    self.board[i] = 0

        return True if p1_empty or p2_empty else False
    
    def utility(self, state, player):
        if player == 1:
            return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
        else:
            return self.board[self.p2_mancala_index] - self.board[self.p1_mancala_index]
        
    def play_with_undo(self, pit):
        """
        Play a move and return the state needed to undo it.
        Returns: dict with undo information, or None if invalid move
        """
        if pit is None or not self.valid_move(pit):
            return None
        
        if self.winning_eval():
            return None
        
        # Save state for undo
        undo_info = {
            'board': self.board.copy(),  # Shallow copy of list is fast
            'current_player': self.current_player,
            'moves_length': len(self.moves)
        }
        
        current_index = self.get_pit_index(pit)
        stones = self.board[current_index]
        self.board[current_index] = 0
        opponent_mancala = self.p2_mancala_index if self.current_player == 1 else self.p1_mancala_index
        
        # Distribute stones
        while stones > 0:
            current_index = (current_index + 1) % len(self.board)
            if current_index == opponent_mancala:
                continue
            self.board[current_index] += 1
            stones -= 1
        
        # Capture stones
        self.capture_stones(current_index)
        self.moves.append((self.current_player, pit))
        self.switch_player()
        
        return undo_info

    def undo_move(self, undo_info):
        """
        Undo a move using the saved undo information.
        """
        if undo_info is None:
            return
        
        self.board = undo_info['board']
        self.current_player = undo_info['current_player']
        
        # Remove moves that were added
        while len(self.moves) > undo_info['moves_length']:
            self.moves.pop()

## Random player vs random player

In [8]:
from utilities.MancalaGame import Mancala
import numpy as np
import random
# from tqdm import tqdm

random_player = [1, 2]

total_games = 100

p1_turns = []
p2_turns = []


wins = {
    "p1": 0,
    "p2": 0,
    "draw": 0,
    "wins_first": 0
}

for play in (range(total_games)):

    game = Mancala(pits_per_player=6, stones_per_pit=4, print_output=False)

    # intialize random player to go first
    game.current_player = random.choice(random_player)
    game.first = game.current_player

    if game.print_output:
        print(
            f'{color.BOLD + color.UNDERLINE + color.GREEN}START GAME #{play+1}{color.END}')
    i = 0

    while not game.winning_eval():
        if game.print_output:
            print(f'{color.BOLD}Turn #{i+1}{color.END}')

        if game.current_player == 1:
            move = game.random_move_generator()
            game.play(move)

            if(game.print_output):
                game.display_board()
        else:
            game.play(game.random_move_generator())
            if (game.print_output):
                game.display_board()
            i += 1
    game.check_win()
    p1_turns.append(len([x[0] for x in game.moves if x[0] == 1]))
    p2_turns.append(len([x[0] for x in game.moves if x[0] == 2]))

    wins["p1"] += game.p1_win
    wins["p2"] += game.p2_win
    wins["draw"] += game.draw
    wins["wins_first"] += game.wins_w_first

print("\n")
stat_title("PLAYER 1 STATS", 12)
list_stat("P1 win %:", f"{round((wins["p1"]/total_games)*100)}%", 29)
list_stat("P1 loss %:", f"{round((wins["p2"]/total_games)*100)}%", 28)
list_stat("Avg turns per game:", f"{round(np.average(p1_turns))}", 19)

stat_title("PLAYER 2 STATS", 12)
list_stat("P2 win %:", f"{round((wins["p2"]/total_games)*100)}%", 29)
list_stat("P2 loss %:", f"{round((wins["p1"]/total_games)*100)}%", 28)
list_stat("Avg turns per game:", f"{round(np.average(p2_turns))}", 19)

stat_title("GAME STATS", 14)
list_stat("Draw %:", f"{round((wins["draw"]/total_games)*100)}%", 31)
list_stat("First Turn Advantage %:", f"{round((wins["wins_first"]/total_games)*100)}%", 15)




⊰ΞΞΞΞΞΞΞΞΞΞΞΞPLAYER 1 STATSΞΞΞΞΞΞΞΞΞΞΞΞ⊱
P1 win %:                          50%
P1 loss %:                         44%
Avg turns per game:                 22
⊰ΞΞΞΞΞΞΞΞΞΞΞΞPLAYER 2 STATSΞΞΞΞΞΞΞΞΞΞΞΞ⊱
P2 win %:                          44%
P2 loss %:                         50%
Avg turns per game:                 22
⊰ΞΞΞΞΞΞΞΞΞΞΞΞΞΞGAME STATSΞΞΞΞΞΞΞΞΞΞΞΞΞΞ⊱
Draw %:                             6%
First Turn Advantage %:            47%


## Minmax vs Random

In [9]:
from utilities.MancalaGame import Mancala
import numpy as np
import random
from copy import deepcopy
from tqdm import tqdm

random_player = [1, 2]

total_games = 100

p1_turns = []
p2_turns = []


wins = {
    "p1": 0,
    "p2": 0,
    "draw": 0,
    "wins_first": 0
}

for play in tqdm(range(total_games)):

    game = Mancala(pits_per_player=6, stones_per_pit=4, print_output=False)

    # intialize random player to go first
    game.current_player = random.choice(random_player)
    game.first = game.current_player

    if game.print_output:
        print(
            f'{color.BOLD + color.UNDERLINE + color.GREEN}START GAME #{play+1}{color.END}')
    i = 0

    while not game.winning_eval():
        if game.print_output:
            print(f'{color.BOLD}Turn #{i+1}{color.END}')

        if game.current_player == 1:
            move = minmax_decision(deepcopy(game), depth=5)
            game.play(move)

            if(game.print_output):
                game.display_board()
        else:
            game.play(game.random_move_generator())
            if (game.print_output):
                game.display_board()
            i += 1
    game.check_win()
    p1_turns.append(len([x[0] for x in game.moves if x[0] == 1]))
    p2_turns.append(len([x[0] for x in game.moves if x[0] == 2]))

    wins["p1"] += game.p1_win
    wins["p2"] += game.p2_win
    wins["draw"] += game.draw
    wins["wins_first"] += game.wins_w_first

print("\n")
stat_title("PLAYER 1 STATS", 12)
list_stat("P1 win %:", f"{round((wins["p1"]/total_games)*100)}%", 29)
list_stat("P1 loss %:", f"{round((wins["p2"]/total_games)*100)}%", 28)
list_stat("Avg turns per game:", f"{round(np.average(p1_turns))}", 19)

stat_title("PLAYER 2 STATS", 12)
list_stat("P2 win %:", f"{round((wins["p2"]/total_games)*100)}%", 29)
list_stat("P2 loss %:", f"{round((wins["p1"]/total_games)*100)}%", 28)
list_stat("Avg turns per game:", f"{round(np.average(p2_turns))}", 19)

stat_title("GAME STATS", 14)
list_stat("Draw %:", f"{round((wins["draw"]/total_games)*100)}%", 31)
list_stat("First Turn Advantage %:", f"{round((wins["wins_first"]/total_games)*100)}%", 15)


100%|██████████| 100/100 [00:05<00:00, 19.70it/s]



⊰ΞΞΞΞΞΞΞΞΞΞΞΞPLAYER 1 STATSΞΞΞΞΞΞΞΞΞΞΞΞ⊱
P1 win %:                          96%
P1 loss %:                          3%
Avg turns per game:                 16
⊰ΞΞΞΞΞΞΞΞΞΞΞΞPLAYER 2 STATSΞΞΞΞΞΞΞΞΞΞΞΞ⊱
P2 win %:                           3%
P2 loss %:                         96%
Avg turns per game:                 16
⊰ΞΞΞΞΞΞΞΞΞΞΞΞΞΞGAME STATSΞΞΞΞΞΞΞΞΞΞΞΞΞΞ⊱
Draw %:                             1%
First Turn Advantage %:            49%


## Alpha-Beta vs Random

In [10]:
from utilities.MancalaGame import Mancala
import numpy as np
import random
from copy import deepcopy
from tqdm import tqdm

random_player = [1, 2]

total_games = 100

p1_turns = []
p2_turns = []


wins = {
    "p1": 0,
    "p2": 0,
    "draw": 0,
    "wins_first": 0
}

for play in tqdm(range(total_games)):

    game = Mancala(pits_per_player=6, stones_per_pit=4, print_output=False)

    # intialize random player to go first
    game.current_player = random.choice(random_player)
    game.first = game.current_player

    if game.print_output:
        print(
            f'{color.BOLD + color.UNDERLINE + color.GREEN}START GAME #{play+1}{color.END}')
    i = 0

    while not game.winning_eval():
        if game.print_output:
            print(f'{color.BOLD}Turn #{i+1}{color.END}')

        if game.current_player == 1:
            move = alpha_beta_search(deepcopy(game), depth=5)
            game.play(move)

            if(game.print_output):
                game.display_board()
        else:
            game.play(game.random_move_generator())
            if (game.print_output):
                game.display_board()
            i += 1
    game.check_win()
    p1_turns.append(len([x[0] for x in game.moves if x[0] == 1]))
    p2_turns.append(len([x[0] for x in game.moves if x[0] == 2]))

    wins["p1"] += game.p1_win
    wins["p2"] += game.p2_win
    wins["draw"] += game.draw
    wins["wins_first"] += game.wins_w_first

print("\n")
stat_title("PLAYER 1 STATS", 12)
list_stat("P1 win %:", f"{round((wins["p1"]/total_games)*100)}%", 29)
list_stat("P1 loss %:", f"{round((wins["p2"]/total_games)*100)}%", 28)
list_stat("Avg turns per game:", f"{round(np.average(p1_turns))}", 19)

stat_title("PLAYER 2 STATS", 12)
list_stat("P2 win %:", f"{round((wins["p2"]/total_games)*100)}%", 29)
list_stat("P2 loss %:", f"{round((wins["p1"]/total_games)*100)}%", 28)
list_stat("Avg turns per game:", f"{round(np.average(p2_turns))}", 19)

stat_title("GAME STATS", 14)
list_stat("Draw %:", f"{round((wins["draw"]/total_games)*100)}%", 31)
list_stat("First Turn Advantage %:", f"{round((wins["wins_first"]/total_games)*100)}%", 15)


100%|██████████| 100/100 [00:01<00:00, 72.01it/s]



⊰ΞΞΞΞΞΞΞΞΞΞΞΞPLAYER 1 STATSΞΞΞΞΞΞΞΞΞΞΞΞ⊱
P1 win %:                          98%
P1 loss %:                          2%
Avg turns per game:                 15
⊰ΞΞΞΞΞΞΞΞΞΞΞΞPLAYER 2 STATSΞΞΞΞΞΞΞΞΞΞΞΞ⊱
P2 win %:                           2%
P2 loss %:                         98%
Avg turns per game:                 16
⊰ΞΞΞΞΞΞΞΞΞΞΞΞΞΞGAME STATSΞΞΞΞΞΞΞΞΞΞΞΞΞΞ⊱
Draw %:                             0%
First Turn Advantage %:            48%
